In [1]:
# -*- coding: utf-8 -*-

# 이 셀은 노트북에서 가장 먼저 실행하는 공통 준비 셀입니다.
# 경로, 클래스, 시드, 기본 상수를 한 곳에서 관리하기 위해 분리했습니다.

from pathlib import Path
import json
import random
import os

import numpy as np

# 1) 데이터 루트 경로를 설정합니다.
#    AI Stages 환경에서 사용자가 쓰던 경로 패턴을 그대로 맞췄습니다.
FULL_DATA_ROOT = Path("/data/ephemeral/home/model/dataset")

# 2) 원본 train.json 경로를 지정합니다.
#    train/val split을 새로 만들기 위해 필요합니다.
TRAIN_JSON = FULL_DATA_ROOT / "train.json"

# 3) test.json 경로를 지정합니다.
#    추론 시 샘플 제출 파일을 만들 때 필요합니다.
TEST_JSON = FULL_DATA_ROOT / "test.json"

# 4) sample_submission 폴더 경로를 지정합니다.
SAMPLE_SUB_DIR = Path("/data/ephemeral/home/model/sample_submission")

# 5) 실험 결과가 저장될 work_dirs 루트를 지정합니다.
WORK_DIR_ROOT = Path("/data/ephemeral/home/model/work_dirs_single")

# 6) 이번 실험의 모델 이름(폴더명)입니다.
#    학습/추론 셀에서 반드시 동일하게 사용해야 체크포인트를 찾습니다.
EXP_NAME = "sparse_rcnn_r50_1024_stage_aug14"

# 7) 입력 이미지 크기를 통일해서 실험 변수를 줄입니다.
#    사용자의 기존 1024 전략을 유지합니다.
IMAGE_SIZE = (1024, 1024)

# 8) 클래스 정의를 대회 포맷에 맞춰 고정합니다.
CLASSES = (
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
)

# 9) 재현성을 위해 시드를 고정합니다.
SEED = 42

def seed_everything(seed: int = 42):
    # 파이썬 기본 랜덤 시드 고정
    random.seed(seed)
    # 넘파이 랜덤 시드 고정
    np.random.seed(seed)
    # 해시 기반 연산 시드 고정
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)

# 10) 경로 존재 여부를 사전에 확인해 불필요한 런타임 오류를 줄입니다.
assert TRAIN_JSON.exists(), f"train.json 없음: {TRAIN_JSON}"
assert TEST_JSON.exists(), f"test.json 없음: {TEST_JSON}"
assert SAMPLE_SUB_DIR.exists(), f"sample_submission 폴더 없음: {SAMPLE_SUB_DIR}"

print("준비 완료")
print("EXP_NAME:", EXP_NAME)
print("IMAGE_SIZE:", IMAGE_SIZE)


준비 완료
EXP_NAME: sparse_rcnn_r50_1024_stage_aug14
IMAGE_SIZE: (1024, 1024)


In [2]:
# -*- coding: utf-8 -*-

# 이 셀은 train.json을 읽어서 원하는 비율로 train/val split 파일을 생성합니다.
# 사용자가 원한 0.88/0.12를 기본값으로 두되,
# split_ratio만 바꾸면 다른 비율도 바로 실험할 수 있게 했습니다.

from pathlib import Path
import json
import random

# 1) split 결과 저장 폴더를 지정합니다.
SPLIT_DIR = FULL_DATA_ROOT / "splits_single"
SPLIT_DIR.mkdir(parents=True, exist_ok=True)

# 2) 기본 비율을 설정합니다.
#    튜닝 안정성과 학습 데이터 확장을 동시에 고려해 0.88을 기본으로 둡니다.
split_ratio = 0.88  # 필요하면 0.85, 0.9 등으로 변경

train_out = SPLIT_DIR / f"train_split_{int(split_ratio*100)}.json"
val_out   = SPLIT_DIR / f"val_split_{int((1-split_ratio)*100)}.json"

# 3) 원본 COCO 형식 train.json을 로드합니다.
with open(TRAIN_JSON, "r") as f:
    coco = json.load(f)

# 4) 이미지 단위로 split 해야 annotation과 매칭이 유지됩니다.
images = coco["images"]
anns = coco["annotations"]

# 5) 이미지 id 목록을 추출합니다.
img_ids = [img["id"] for img in images]

# 6) 시드 고정 상태로 셔플합니다.
random.shuffle(img_ids)

# 7) 분할 지점을 계산합니다.
cut = int(len(img_ids) * split_ratio)

train_ids = set(img_ids[:cut])
val_ids   = set(img_ids[cut:])

# 8) 이미지 리스트를 분리합니다.
train_images = [img for img in images if img["id"] in train_ids]
val_images   = [img for img in images if img["id"] in val_ids]

# 9) annotation도 이미지 id 기준으로 분리합니다.
train_anns = [a for a in anns if a["image_id"] in train_ids]
val_anns   = [a for a in anns if a["image_id"] in val_ids]

# 10) 카테고리/기타 메타는 그대로 유지합니다.
base = {k: v for k, v in coco.items() if k not in ["images", "annotations"]}

train_coco = dict(**base, images=train_images, annotations=train_anns)
val_coco   = dict(**base, images=val_images, annotations=val_anns)

# 11) 저장합니다.
with open(train_out, "w") as f:
    json.dump(train_coco, f)

with open(val_out, "w") as f:
    json.dump(val_coco, f)

print("train split:", train_out)
print("val split  :", val_out)
print("train images:", len(train_images), "val images:", len(val_images))


train split: /data/ephemeral/home/model/dataset/splits_single/train_split_88.json
val split  : /data/ephemeral/home/model/dataset/splits_single/val_split_12.json
train images: 4297 val images: 586


In [ ]:
# -*- coding: utf-8 -*-

# 이 셀은 MMDetection의 base config를 불러와
# 데이터셋 경로, 클래스 수, 학습 epoch,
# 그리고 사용자가 정리한 증강 철학을 반영한 3-스테이지 파이프라인을 적용합니다.
# 마지막으로 W&B 팀 엔티티/프로젝트 설정까지 구성합니다.

from pathlib import Path

from mmengine.config import Config
from mmengine.runner import Runner

from mmdet.utils import register_all_modules

# 1) MMDetection 루트를 지정합니다.
MMD_ROOT = Path("/data/ephemeral/home/model/baseline/mmdetection")

# 2) Sparse R-CNN 기본 config를 선택합니다.
#    사진 목록에 없던 새로운 단일 모델 축을 만들기 위한 선택입니다.
base_cfg_rel = "configs/sparse_rcnn/sparse-rcnn_r50_fpn_300-proposals_crop-ms-480-800-3x_coco.py"
base_cfg_path = MMD_ROOT / base_cfg_rel

assert base_cfg_path.exists(), f"base cfg 없음: {base_cfg_path}"

# 3) split 파일 경로를 위 셀에서 만든 이름 규칙과 맞춥니다.
SPLIT_DIR = FULL_DATA_ROOT / "splits_single"
split_ratio = 0.88  # 셀 1과 동일하게 맞춰야 합니다.

TRAIN_SPLIT = SPLIT_DIR / f"train_split_{int(split_ratio*100)}.json"
VAL_SPLIT   = SPLIT_DIR / f"val_split_{int((1-split_ratio)*100)}.json"

assert TRAIN_SPLIT.exists(), f"train split 없음: {TRAIN_SPLIT}"
assert VAL_SPLIT.exists(), f"val split 없음: {VAL_SPLIT}"

# 4) 모델/데이터 모듈을 등록합니다.
register_all_modules(init_default_scope=True)

# 5) base config를 로드합니다.
cfg = Config.fromfile(str(base_cfg_path))
cfg.default_scope = "mmdet"

# 6) 실험 work_dir을 고정합니다.
#    체크포인트/로그 경로 혼선을 방지합니다.
EXP_WORK_DIR = WORK_DIR_ROOT / EXP_NAME
cfg.work_dir = str(EXP_WORK_DIR)

# 7) 데이터 루트를 대회 데이터 경로로 교체합니다.
cfg.data_root = str(FULL_DATA_ROOT)

# 8) train/val dataloader의 dataset 설정을 가져와 수정합니다.
#    MMDet 버전에 따라 dict 구조가 조금 다르므로 안전하게 접근합니다.
train_loader = cfg.train_dataloader
val_loader = cfg.val_dataloader

train_ds = train_loader.get("dataset", train_loader)
val_ds = val_loader.get("dataset", val_loader)

# 9) 클래스 메타 정보를 강제 설정합니다.
train_ds.metainfo = dict(classes=CLASSES)
val_ds.metainfo = dict(classes=CLASSES)

# 10) annotation 파일 경로를 split json으로 교체합니다.
train_ds.ann_file = str(TRAIN_SPLIT)
val_ds.ann_file   = str(VAL_SPLIT)

# 11) 이미지 경로 prefix를 맞춥니다.
#     대회 데이터 구조에 따라 img=""가 안전합니다.
train_ds.data_prefix = dict(img="")
val_ds.data_prefix   = dict(img="")

# 12) 사용자 전략을 반영한 3-스테이지 파이프라인을 정의합니다.
#     Cascade/RCNN 계열에 맞는 mild~medium 증강 기조를 유지합니다.
train_pipeline_stage1 = [
    dict(type="LoadImageFromFile"),
    dict(type="LoadAnnotations", with_bbox=True),

    # 기본 flip
    dict(type="RandomFlip", prob=0.5),

    # small object 개선을 위한 보수적 MinIoU Random Crop
    dict(
        type="MinIoURandomCrop",
        min_ious=[0.4, 0.5, 0.6],
        min_crop_size=0.6,
    ),

    # 멀티스케일 리사이즈 (과도한 변형 방지 위해 스케일을 3개로 제한)
    dict(
        type="RandomChoiceResize",
        scales=[(800, 800), (896, 896), (1024, 1024)],
        keep_ratio=True,
    ),

    # 색/밝기 기반의 안전한 photometric 증강
    dict(
        type="PhotoMetricDistortion",
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18
    ),

    # CutOut은 초반에만 사용 (과하면 localization 손상 가능)
    dict(
        type="CutOut",
        n_holes=5,
        cutout_shape=[(32, 32), (48, 48), (64, 64)]
    ),

    dict(type="PackDetInputs"),
]

train_pipeline_stage2 = [
    dict(type="LoadImageFromFile"),
    dict(type="LoadAnnotations", with_bbox=True),

    dict(type="RandomFlip", prob=0.5),

    dict(
        type="MinIoURandomCrop",
        min_ious=[0.4, 0.5, 0.6],
        min_crop_size=0.6,
    ),

    dict(
        type="RandomChoiceResize",
        scales=[(800, 800), (896, 896), (1024, 1024)],
        keep_ratio=True,
    ),

    dict(
        type="PhotoMetricDistortion",
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18
    ),

    # Stage 2에서는 CutOut을 제거해 박스 품질 저하를 줄입니다.
    dict(type="PackDetInputs"),
]

train_pipeline_stage3 = [
    dict(type="LoadImageFromFile"),
    dict(type="LoadAnnotations", with_bbox=True),

    # 후반에는 inference와 가까운 분포로 수렴시키기 위해 단순화합니다.
    dict(type="RandomFlip", prob=0.5),

    # 최종 크기 고정
    dict(type="Resize", scale=IMAGE_SIZE, keep_ratio=True),

    # 색 증강도 약하게 유지하거나 필요 시 제거 가능
    dict(
        type="PhotoMetricDistortion",
        brightness_delta=16,
        contrast_range=(0.8, 1.2),
        saturation_range=(0.8, 1.2),
        hue_delta=10
    ),

    dict(type="PackDetInputs"),
]

# 13) 실제 train dataset pipeline을 Stage 1로 초기 설정합니다.
train_ds.pipeline = train_pipeline_stage1

# 14) val pipeline은 안정적인 고정 리사이즈로 통일합니다.
val_ds.pipeline = [
    dict(type="LoadImageFromFile"),
    dict(type="Resize", scale=IMAGE_SIZE, keep_ratio=True),
    dict(type="PackDetInputs"),
]

# 15) 모델의 num_classes를 데이터셋 클래스 수에 맞춥니다.
#     config 내부를 재귀적으로 탐색해 num_classes 키를 찾아 수정합니다.
def set_num_classes(node, num_classes: int):
    if isinstance(node, dict):
        if "num_classes" in node:
            node["num_classes"] = num_classes
        for v in node.values():
            set_num_classes(v, num_classes)
    elif isinstance(node, list):
        for v in node:
            set_num_classes(v, num_classes)

set_num_classes(cfg.model, len(CLASSES))

# 16) epoch 수를 14로 설정합니다.
#     base config가 3x 스케줄일 수 있으므로
#     max_epochs를 직접 덮어써서 실험 목적에 맞춥니다.
cfg.train_cfg.max_epochs = 14

# 17) batch size는 GPU 상황에 맞게 조정 가능합니다.
#     여기서는 안전한 값으로 예시만 둡니다.
cfg.train_dataloader.batch_size = 4
cfg.val_dataloader.batch_size = 2

# 18) 파이프라인 스테이지를 epoch 비율에 따라 바꾸는 Hook을 정의합니다.
from mmengine.hooks import Hook

# -*- coding: utf-8 -*-
# -*- coding: utf-8 -*-

from mmengine.hooks import Hook
from mmengine.registry import HOOKS

# 같은 이름으로 여러 번 등록될 때 충돌/캐시 문제가 생길 수 있어
# 클래스 이름을 V2로 바꿔 새 엔트리로 등록합니다.

@HOOKS.register_module()
class PipelineSwitchHookV2(Hook):
    # stage1_end, stage2_end를 명시적으로 받되
    # 혹시 모를 추가 인자 유입까지 안전하게 처리하기 위해 **kwargs도 받습니다.
    def __init__(self, stage1_end=0.4, stage2_end=0.75, **kwargs):
        self.stage1_end = stage1_end
        self.stage2_end = stage2_end

    def before_train_epoch(self, runner):
        # 현재 epoch
        epoch = runner.epoch
        # 전체 학습 epoch
        max_epochs = runner.max_epochs
        # 진행 비율
        ratio = (epoch + 1) / max_epochs

        # 실제 Dataset 객체 접근
        ds = runner.train_dataloader.dataset

        # 1단계: 강/중 증강
        if ratio <= self.stage1_end:
            ds.pipeline = train_pipeline_stage1
            runner.logger.info(
                f"[PipelineSwitch] Stage1 pipeline 적용 (epoch={epoch+1}/{max_epochs})"
            )

        # 2단계: 중간 강도
        elif ratio <= self.stage2_end:
            ds.pipeline = train_pipeline_stage2
            runner.logger.info(
                f"[PipelineSwitch] Stage2 pipeline 적용 (epoch={epoch+1}/{max_epochs})"
            )

        # 3단계: 약한 증강(추론 분포 근접)
        else:
            ds.pipeline = train_pipeline_stage3
            runner.logger.info(
                f"[PipelineSwitch] Stage3 pipeline 적용 (epoch={epoch+1}/{max_epochs})"
            )


# 기존에 cfg.custom_hooks.append(dict(type=PipelineSwitchHook)) 를 썼다면 삭제/교체
# -*- coding: utf-8 -*-

# 기존에 PipelineSwitchHook을 넣은 줄이 있으면 제거하고 아래만 남깁니다.

cfg.custom_hooks = cfg.get("custom_hooks", [])

cfg.custom_hooks.append(
    dict(
        type="PipelineSwitchHookV2",  # 새 이름으로 지정
        stage1_end=0.4,
        stage2_end=0.75
    )
)


# 20) W&B 설정
#     핵심은 entity를 '조직'이 아니라 '팀'으로 지정하는 것입니다.
ENTITY = "boostcamp8-cv-08"
PROJECT = "kim"

cfg.visualizer = dict(
    type="DetLocalVisualizer",
    vis_backends=[
        dict(type="LocalVisBackend"),
        dict(
            type="WandbVisBackend",
            init_kwargs=dict(
                entity=ENTITY,
                project=PROJECT,
                name=EXP_NAME,
            ),
        ),
    ],
)

# 22) 로그 주기를 적당히 조절합니다.
cfg.default_hooks = cfg.get("default_hooks", {})
cfg.default_hooks["logger"] = dict(type="LoggerHook", interval=50)

print("Config 준비 완료")
print("base cfg:", base_cfg_path)
print("work_dir:", cfg.work_dir)
print("train ann:", train_ds.ann_file)
print("val ann  :", val_ds.ann_file)
print("max_epochs:", cfg.train_cfg.max_epochs)
print("wandb entity/project:", ENTITY, "/", PROJECT)


Config 준비 완료
base cfg: /data/ephemeral/home/model/baseline/mmdetection/configs/sparse_rcnn/sparse-rcnn_r50_fpn_300-proposals_crop-ms-480-800-3x_coco.py
work_dir: /data/ephemeral/home/model/work_dirs_single/sparse_rcnn_r50_1024_stage_aug14
train ann: /data/ephemeral/home/model/dataset/splits_single/train_split_88.json
val ann  : /data/ephemeral/home/model/dataset/splits_single/val_split_12.json
max_epochs: 14
wandb entity/project: boostcamp8-cv-08 / kim


In [4]:
# -*- coding: utf-8 -*-

# 이 셀은 위에서 구성한 cfg로 학습을 시작합니다.
# W&B 로그인은 터미널에서 이미 해두었다고 가정합니다.

from mmengine.runner import Runner

# 1) Runner를 config로 생성합니다.
runner = Runner.from_cfg(cfg)

# 2) 학습을 시작합니다.
runner.train()

print("학습 완료")
print("체크포인트 폴더:", cfg.work_dir)


12/08 08:18:46 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1608637542
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: n/a
    PyTorch: 2.1.2+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_7

/bin/sh: 1: gcc: not found


12/08 08:18:47 - mmengine - INFO - Config:
auto_scale_lr = dict(base_batch_size=16, enable=False)
backend_args = None
custom_hooks = [
    dict(stage1_end=0.4, stage2_end=0.75, type='PipelineSwitchHook'),
]
data_root = '/data/ephemeral/home/model/dataset'
dataset_type = 'CocoDataset'
default_hooks = dict(
    checkpoint=dict(interval=1, type='CheckpointHook'),
    logger=dict(interval=50, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='DetVisualizationHook'))
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
load_from = None
log_level = 'INFO'
log_processor = dict(by_epoch=True, type='LogProcessor', window_size=50)
max_epochs = 36
model = dict(
    backbone=dict(
        depth=50,
        frozen_stages=1,
        init_cfg=dict(check

wandb: Currently logged in as: hae24923 (boostcamp8-cv-08) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


12/08 08:18:53 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.


TypeError: PipelineSwitchHook.__init__() got an unexpected keyword argument 'stage1_end'

In [1]:
import wandb

# 팀 엔티티를 반드시 지정합니다.
run = wandb.init(
    entity="boostcamp8-cv-08",
    project="kim",
    name="smoke-test",
)

run.log({"ping": 1})
run.finish()

print("W&B smoke test done")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


ping,▁
ping,1


W&B smoke test done
